<a href="https://colab.research.google.com/github/Vishnucreate/AI_voice_agents-Architecture/blob/main/WhisperAIFineTuningFailednosubtitles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install git+https://github.com/openai/whisper.git
!pip install datasets transformers accelerate torchaudio
!pip install yt-dlp


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-zfgbgujp
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-zfgbgujp
  Resolved https://github.com/openai/whisper.git to commit dd985ac4b90cafeef8712f2998d62c59c3e62d22
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━

In [4]:
# 1. Install dependencies
!pip install nemo_toolkit[asr] yt-dlp srt
!sudo apt install ffmpeg
!pip install yt-dlp srt librosa ffmpeg-python



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [9]:
!yt-dlp -f 'bestaudio[ext=m4a]' --write-subs --sub-lang en --convert-subs srt -o 'video1.%(ext)s' https://www.youtube.com/watch?v=YBzWTIexszQ
!yt-dlp -f 'bestaudio[ext=m4a]' --write-subs --sub-lang en --convert-subs srt \
  -o 'commentary1.%(ext)s' https://www.youtube.com/watch?v=REPLACE_WITH_NEW_ID



[youtube] Extracting URL: https://www.youtube.com/watch?v=YBzWTIexszQ
[youtube] YBzWTIexszQ: Downloading webpage
[youtube] YBzWTIexszQ: Downloading tv client config
[youtube] YBzWTIexszQ: Downloading tv player API JSON
[youtube] YBzWTIexszQ: Downloading ios player API JSON
[youtube] YBzWTIexszQ: Downloading m3u8 information
[info] YBzWTIexszQ: Downloading subtitles: en
[info] YBzWTIexszQ: Downloading 1 format(s): 140
[info] Writing video subtitles to: video1.en.vtt
[download] Destination: video1.en.vtt
[download] 100% of    556.00B in 00:00:00 at 2.90KiB/s
[SubtitlesConvertor] Converting subtitles
Deleting original file video1.en.vtt (pass -k to keep)
[download] video1.m4a has already been downloaded
[download] 100% of    3.22MiB
[youtube] Extracting URL: https://www.youtube.com/watch?v=REPLACE_WITH_NEW_ID
[youtube] REPLACE_WIT: Downloading webpage
[youtube] REPLACE_WIT: Downloading tv client config
[youtube] REPLACE_WIT: Downloading tv player API JSON
[youtube] REPLACE_WIT: Downloadin

In [8]:
import ffmpeg

def convert_to_wav(input_path, output_path):
    ffmpeg.input(input_path).output(output_path, acodec='pcm_s16le', ac=1, ar='16000').run()

convert_to_wav("video1.m4a", "video1.wav")
convert_to_wav("video2.m4a", "video2.wav")


Error: ffmpeg error (see stderr output for detail)

In [3]:
import srt, json

def build_manifest(video):
    with open(f"{video}.en.srt") as f, open(f"{video}_manifest.json","w") as out:
        subs = list(srt.parse(f.read()))
        for sub in subs:
            dur = round((sub.end - sub.start).total_seconds(),2)
            if dur > 0.5:
                out.write(json.dumps({
                    "audio_filepath": f"{video}.wav",
                    "offset": round(sub.start.total_seconds(),2),
                    "duration": dur,
                    "text": sub.content.replace("\n"," ")
                }) + "\n")

build_manifest("video1")
build_manifest("video2")


FileNotFoundError: [Errno 2] No such file or directory: 'video2.en.srt'